In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import plot_model 
from tqdm import tqdm

# Hyperparameter
img_path_train = 'seg_train/seg_train'
img_path_test = 'seg_test/seg_test'
class_name = ['buildings',
'forest',
'glacier',
'mountain',
'sea',
'street']
IMG_SIZE = 150
INPUT_SHAPE = (150,150,3) 
BATCH_SIZE = 128
EPOCHS = 20

In [5]:
class_name_label = {class_name:i for i, class_name in enumerate(class_name)}
print(class_name_label)
print(len(class_name_label))


def get_data():
    data = [img_path_train, img_path_test]
    output = []

    for dataset in data:
        images = []
        labels = []
        print("Loading dataset:", dataset)
        
        for folder in os.listdir(dataset):
            if folder not in class_name_label:
                print(f"Skipping unknown folder: {folder}")
                continue
            
            label = class_name_label[folder]
            folder_path = os.path.join(dataset, folder)
            
            for file in tqdm(os.listdir(folder_path), desc=f"Processing {folder}"):
                img_path = os.path.join(folder_path, file)
                
                # Read & Resize img
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Warning: Could not read {img_path}. Skipping.")
                    continue
                
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                
                # Append photos
                images.append(img)
                labels.append(label)
        
        # NumPy
        images = np.array(images, dtype='float32') / 255.0
        labels = np.array(labels, dtype='int32')
        output.append((images, labels))
    
    return output

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
6


In [6]:
(train_images, train_labels), (test_images, test_labels) = get_data()

print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)


Loading dataset: seg_train/seg_train


Processing buildings:   0%|          | 0/2191 [00:00<?, ?it/s]

Processing street: 100%|██████████| 2382/2382 [00:04<00:00, 562.63it/s]


Loading dataset: seg_test/seg_test


Processing street: 100%|██████████| 501/501 [00:01<00:00, 489.99it/s]


Train Images Shape: (14034, 150, 150, 3)
Train Labels Shape: (14034,)
Test Images Shape: (3000, 150, 150, 3)
Test Labels Shape: (3000,)


In [7]:
num_train_images = train_images.shape[0]
train_images_flat = train_images.reshape(num_train_images, -1)  # Aplana las imágenes

In [ ]:
from sklearn.cluster import KMeans

# Experimentar con diferentes valores de k
inertia = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(train_images_flat)
    inertia.append(kmeans.inertia_)

# Visualizar la inercia para determinar el número óptimo de clusters
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o')
plt.title('Inercia vs Número de Clusters (K)')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inercia')
plt.xticks(k_values)
plt.grid()
plt.show()

In [8]:
from sklearn.cluster import KMeans

# Aplicar K-means y predecir los clusters
kmeans = KMeans(n_clusters=6, random_state=42)  # Usando 6 como ejemplo
train_labels_predicted = kmeans.fit_predict(train_images_flat)

# Visualizar los primeros dos componentes principales
plt.figure(figsize=(10, 10))
plt.scatter(train_images_pca[:, 0], train_images_pca[:, 1], c=train_labels_predicted, cmap='viridis', alpha=0.5)
plt.title('Visualización de K-means en el espacio PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.colorbar(label='Cluster')
plt.show()


KeyboardInterrupt: 

In [9]:
def model_CNN():
    model = Sequential([

        Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        MaxPool2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        
        Flatten(),
        
        Dense(128, activation='relu'),
        
        Dense(6, activation='softmax')
    ])
    return model


model = model_CNN()
model.summary()

# Mostrar modelo
os.makedirs('plots', exist_ok=True)
plot_model(model, to_file='plots/model_plot.jpg', show_shapes=True)


model.compile(
    optimizer= 'adam',  
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Entrenar
history = model.fit(train_images, train_labels, batch_size = BATCH_SIZE, epochs = EPOCHS)

c:\Users\feded\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    10,616,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,637,126 (40.58 MB)

 Trainable params: 10,637,126 (40.58 MB)

 Non-trainable params: 0 (0.00 B)

You must install pydot (`pip install pydot`) for `plot_model` to work.
Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 362s 3s/step - accuracy: 0.3740 - loss: 2.4068
Epoch 2/20
 27/110 ━━━━━━━━━━━━━━━━━━━━ 4:23 3s/step - accuracy: 0.7051 - loss: 0.8490

KeyboardInterrupt: 

In [1]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Test
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
# Predict
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(test_labels, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_name, yticklabels=class_name)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Ploting
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'model' is not defined

In [ ]:
import random

def test_single_image(index=None):
    if index is None:
        index = random.randint(0, len(test_images) - 1)
    
    img = test_images[index]
    true_label = test_labels[index]
    
    # pred
    pred = model.predict(np.expand_dims(img, axis=0))
    predicted_class = np.argmax(pred)

    
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title(f"True: {class_name[true_label]}, Predicted: {class_name[predicted_class]}")
    plt.show()

test_single_image()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# 1. Evaluar el modelo
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 2. Hacer predicciones
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)



El principal problema en mi caso es la falta de un equipo capaz de procesar esta cantidad de datos, un error que podria pasar en mas de una imagen por ejemplo cuando en el fondo de una ciudad aparece una montaña y tome como otra categoria, esto podemos solucionar un poco, ajustando las metricas sin pasarnos para que el programa vea los patrones y no aprenda del ejemplo